In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from nltk.tokenize import word_tokenize
from nltk import classify

In [2]:
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

In [3]:
musical_reviews = pd.read_csv('F:\\Kaggle Datasets\\573774_1043123_bundle_archive\\Musical_instruments_reviews.csv')

In [4]:
musical_reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [5]:
musical_reviews.overall.value_counts()

5.0    6938
4.0    2084
3.0     772
2.0     250
1.0     217
Name: overall, dtype: int64

In [6]:
musical_reviews.isna().sum()

reviewerID         0
asin               0
reviewerName      27
helpful            0
reviewText         7
overall            0
summary            0
unixReviewTime     0
reviewTime         0
dtype: int64

In [7]:
musical_reviews.reviewText .fillna("",inplace = True)

In [8]:
musical_reviews.isna().sum()

reviewerID         0
asin               0
reviewerName      27
helpful            0
reviewText         0
overall            0
summary            0
unixReviewTime     0
reviewTime         0
dtype: int64

In [9]:
musical_reviews_0 = musical_reviews.drop(["reviewerID","asin","reviewerName","helpful","unixReviewTime","reviewTime"],axis=1)

In [10]:
musical_reviews_0.head()

,reviewText,overall,summary
0,"Not much to write about here, but it does exac...",5.0,good
1,The product does exactly as it should and is q...,5.0,Jake
2,The primary job of this device is to block the...,5.0,It Does The Job Well
3,Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY
4,This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.


In [11]:
musical_reviews_0['combinedText'] = musical_reviews_0['reviewText'] + ' ' + musical_reviews_0['summary']

In [12]:
musical_reviews_0.head()

,reviewText,overall,summary,combinedText
0,"Not much to write about here, but it does exac...",5.0,good,"Not much to write about here, but it does exac..."
1,The product does exactly as it should and is q...,5.0,Jake,The product does exactly as it should and is q...
2,The primary job of this device is to block the...,5.0,It Does The Job Well,The primary job of this device is to block the...
3,Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,Nice windscreen protects my MXL mic and preven...
4,This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,This pop filter is great. It looks and perform...


In [13]:
musical_reviews_0 = musical_reviews_0.drop(["reviewText","summary"],axis=1)

In [14]:
musical_reviews_0.head()

,overall,combinedText
0,5.0,"Not much to write about here, but it does exac..."
1,5.0,The product does exactly as it should and is q...
2,5.0,The primary job of this device is to block the...
3,5.0,Nice windscreen protects my MXL mic and preven...
4,5.0,This pop filter is great. It looks and perform...


In [15]:
def apply_encoding(x):
    if(x ==1 or x==2 or x==3):
        return 0
    else:
        return 1

In [16]:
musical_reviews_0.overall = musical_reviews_0.overall.apply(apply_encoding)

In [17]:
musical_reviews_0.overall.value_counts()

1    9022
0    1239
Name: overall, dtype: int64

In [18]:
musical_reviews_0["combinedText"] = musical_reviews_0.apply(lambda row: nltk.word_tokenize(row['combinedText']), axis=1)

In [19]:
musical_reviews_0.head()
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [20]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [21]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [22]:
def remove_noise(text):
    clean_text = []
    for token in text:
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        token = lemmatizer.lemmatize(token.strip(),get_simple_pos(token))
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            clean_text.append(token.lower())
    return clean_text

In [23]:
musical_reviews_0.combinedText = musical_reviews_0.combinedText.apply(remove_noise)

In [24]:
musical_reviews_0.head(20)

,overall,combinedText
0,1,"[much, write, doe, exactly, 's, supposed, filt..."
1,1,"[product, doe, exactly, quite, affordable.i, r..."
2,1,"[primary, job, device, block, breath, would, o..."
3,1,"[nice, windscreen, protects, mxl, mic, prevent..."
4,1,"[pop, filter, great, look, performs, like, stu..."
5,1,"[good, bought, another, one, love, heavy, cord..."
6,1,"[used, monster, cable, year, good, reason, lif..."
7,0,"[use, cable, run, output, pedal, chain, input,..."
8,1,"[perfect, epiphone, sheraton, ii, monster, cab..."
9,1,"[monster, make, best, cable, lifetime, warrant..."


In [25]:
musical_reviews_0.combinedText = [''.join(str(item)) for item in musical_reviews_0.combinedText.values]

In [26]:
x_train,x_test,y_train,y_test = train_test_split(musical_reviews_0.combinedText,musical_reviews_0.overall,test_size = 0.2 , random_state = 0)

In [27]:
cv=CountVectorizer()
cv_train_reviews=cv.fit_transform(x_train)
cv_test_reviews=cv.transform(x_test)

In [29]:
lr=LogisticRegression(max_iter=500)
lr_bow=lr.fit(cv_train_reviews,y_train)

In [30]:
lr_bow_predict=lr.predict(cv_test_reviews)

In [33]:
lr_bow_score=accuracy_score(y_test,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)

lr_bow_score : 0.8986848514369216


In [40]:
lr_bow_report=classification_report(y_test,lr_bow_predict,target_names=['0','1'])
print(lr_bow_report)

              precision    recall  f1-score   support

           0       0.56      0.43      0.48       228
           1       0.93      0.96      0.94      1825

    accuracy                           0.90      2053
   macro avg       0.74      0.69      0.71      2053
weighted avg       0.89      0.90      0.89      2053

